In [1]:
import numpy as np
import matplotlib.pyplot as plt
from tqdm import trange
from flygym.mujoco import Parameters
from flygym.mujoco.examples.turning_controller import HybridTurningNMF
from flygym.mujoco.arena import OdorArena

# Odor source: array of shape (num_odor_sources, 3) - xyz coords of odor sources
odor_source = np.array([[24, 0, 1.5], [8, -4, 1.5]]) #, [16, 4, 1.5]])

# Peak intensities: array of shape (num_odor_sources, odor_dimesions)
# For each odor source, if the intensity is (x, 0) then the odor is in the 1st dimension
# (in this case attractive). If it's (0, x) then it's in the 2nd dimension (in this case
# aversive)
peak_intensity = np.array([[1, 0], [0, 1]]) #, [0, 1]])

# Marker colors: array of shape (num_odor_sources, 4) - RGBA values for each marker,
# normalized to [0, 1]
#marker_colors = [[255, 127, 14], [31, 119, 180], [31, 119, 180]]
#marker_colors = np.array([[*np.array(color) / 255, 1] for color in marker_colors])

odor_dimesions = len(peak_intensity[0])

odor_valence = [1,2]

In [2]:
arena = OdorArena(
     odor_source=odor_source,
     peak_intensity=peak_intensity,
     odor_valence=odor_valence,
     diffuse_func=lambda x: x**-2,
     marker_size=0.3,
 )

In [3]:
def is_yeast(source_index) -> bool:
        """This function returns whether the food 
        source is yeast or sucrose"""
        if (
            arena.peak_odor_intensity[source_index][0]
            > arena.peak_odor_intensity[source_index][1]
        ):
            return True
        else:
            return False

In [4]:
def compute_richest_yeast_source() -> float:
    arena_valence_dict = arena.valence_dictionary
    found_key = True
    while found_key:
        max_key = max(arena_valence_dict, key=arena_valence_dict.get)
        if max_key<0 and is_yeast:
            arena_valence_dict.pop(max_key)
            print(arena_valence_dict)
        else: 
            found_key = False
    for el in range(len(arena.peak_odor_intensity)):
        if max_key == arena.compute_smell_key_value(arena.peak_odor_intensity[el]):
            return el


In [5]:
compute_richest_yeast_source()

{1: 1}


0

In [6]:
# Add the NeuroMechFLy

contact_sensor_placements = [
    f"{leg}{segment}"
    for leg in ["LF", "LM", "LH", "RF", "RM", "RH"]
    for segment in ["Tibia", "Tarsus1", "Tarsus2", "Tarsus3", "Tarsus4", "Tarsus5"]
]
sim_params = Parameters(
    timestep=1e-4,
    render_mode="saved",
    render_playspeed=0.5,
    render_window_size=(800, 608),
    enable_olfaction=True,
    enable_adhesion=True,
    draw_adhesion=False,
    render_camera="birdeye_cam",
)
sim = HybridTurningNMF(
    sim_params=sim_params,
    arena=arena,
    spawn_pos=(0, 0, 0.2),
    contact_sensor_placements=contact_sensor_placements,
    food_loss_rate = 0.00003,
)

In [7]:
decision_interval = 0.01
run_time = sim.simulation_time
num_decision_steps = int(run_time / decision_interval)
print("Number of decision steps: ", num_decision_steps)
physics_steps_per_decision_step = int(decision_interval / sim_params.timestep)
print("Physics steps per decision step: ", physics_steps_per_decision_step)

obs_hist = []
odor_history = []
obs, _ = sim.reset()

Number of decision steps:  500
Physics steps per decision step:  100


In [8]:
sim.compute_closest_yeast_source(obs)

0

In [10]:
# Bool to see if simulation is done
sim_end = False
# Get fly's hunger state
hunger_limits = sim.food_requirements
# To keep track of the previous internal state
prev_int_state = ""
# Intialize gains
attractive_gain = 0
aversive_gain = 0

#mating_state = sim.compute_mating_state()
mating_state = sim.mating_state
# Run simulation
# For each decision step
for i in trange(num_decision_steps):
    if not sim_end:
        # Check where the fly's hunger level is at
        int_state = sim.compute_internal_state()
        # Change gains only if the internal state has changed
        if int_state != prev_int_state:
            print("Internal state changed to: ", int_state)
            mating_state = "mated"
            att_gain, av_gain = arena.generate_random_gains_food_internal_state(int_state, mating_state)
            attractive_gain = att_gain
            aversive_gain = av_gain
            prev_int_state = int_state
            print("Attractive gain: ", attractive_gain)
            print("Aversive gain: ", aversive_gain)

100%|██████████| 500/500 [00:00<00:00, 2175468.88it/s]

Internal state changed to:  satiated
Attractive gain:  -397
Aversive gain:  0


In [11]:
def compute_closest_yeast_source(obs) -> float:
        """This function returns the index of the closest 
        yeast source given the current position of the 
        fly in the simulation"""
        distance = np.inf
        index_source = 0
        for i in range(len(sim.arena.odor_source)):
            tmp_distance = np.linalg.norm(
                obs["fly"][0, :2] - sim.arena.odor_source[i, :2]
            )
            if tmp_distance < distance:
                if sim.arena.is_yeast(i):
                    distance = tmp_distance
                    index_source = i
        return index_source

In [12]:
yeast_source = compute_closest_yeast_source(obs)

In [13]:
def generate_exploration_turning_control(attractive_gain, aversive_gain):
    # Compute bias from odor intensity
        attractive_intensities = np.average(
            obs["odor_intensity"][0, :].reshape(2, 2), axis=0, weights=[9, 1]
        )
        aversive_intensities = np.average(
            obs["odor_intensity"][1, :].reshape(2, 2), axis=0, weights=[10, 0]
        )
        attractive_bias = (
            attractive_gain
            * (attractive_intensities[0] - attractive_intensities[1])
            / attractive_intensities.mean()
        )
        aversive_bias = (
            aversive_gain
            * (aversive_intensities[0] - aversive_intensities[1])
            / aversive_intensities.mean()
        )
        effective_bias = aversive_bias + attractive_bias
        effective_bias_norm = np.tanh(effective_bias**2) * np.sign(effective_bias)
        assert np.sign(effective_bias_norm) == np.sign(effective_bias)

        # Compute control signal
        control_signal = np.ones((2,))
        side_to_modulate = int(effective_bias_norm > 0)
        modulation_amount = np.abs(effective_bias_norm) * 0.8
        control_signal[side_to_modulate] -= modulation_amount

In [20]:
def get_specific_olfaction(index_source, sim, arena):
    odors = arena.odor_source[index_source]
    odors = np.expand_dims(odors, axis=0)
    _odor_source_repeated = odors[:, np.newaxis, np.newaxis, :]
    _odor_source_repeated = np.repeat(
        _odor_source_repeated, arena.odor_dimensions, axis=1
    )
    _odor_source_repeated = np.repeat(
        _odor_source_repeated, arena.num_sensors, axis=2
    )
    peak_odor_intesity = arena.peak_odor_intensity[index_source]
    peak_odor_intesity = np.expand_dims(peak_odor_intesity, axis=0)
    _peak_intensity_repeated = peak_odor_intesity[:, :, np.newaxis]
    _peak_intensity_repeated = np.repeat(
        _peak_intensity_repeated, arena.num_sensors, axis=2
    )
    _peak_intensity_repeated = _peak_intensity_repeated
    antennae_pos = sim.physics.bind(sim._antennae_sensors).sensordata
    antennae_pos = antennae_pos.reshape(4,3)
    antennae_pos_repeated = antennae_pos[np.newaxis, np.newaxis, :, :]
    dist_3d = antennae_pos_repeated - _odor_source_repeated  # (n, k, w, 3)
    dist_euc = np.linalg.norm(dist_3d, axis=3)  # (n, k, w)
    scaling = arena.diffuse_func(dist_euc)  # (n, k, w)
    intensity = _peak_intensity_repeated * scaling  # (n, k, w)
    return intensity.sum(axis=0)  # (k, w)


In [25]:
def generate_specific_turning_control(arena, index_source,antennae_pos, attractive_gain):
    # Compute bias from odor intensity knowing that 
    # the fly needs to approach a yeast source specified 
    # by the index_source    
    obs = get_specific_olfaction(index_source, sim, arena)

    attractive_intensities = np.average(
        obs[0, :].reshape(2, 2), axis=0, weights=[9, 1]
    )
    attractive_bias = (
        attractive_gain
        * (attractive_intensities[0] - attractive_intensities[1])
        / attractive_intensities.mean()
    )
    aversive_bias = 0
    effective_bias = aversive_bias + attractive_bias
    effective_bias_norm = np.tanh(effective_bias**2) * np.sign(effective_bias)
    assert np.sign(effective_bias_norm) == np.sign(effective_bias)

    # Compute control signal
    control_signal = np.ones((2,))
    side_to_modulate = int(effective_bias_norm > 0)
    modulation_amount = np.abs(effective_bias_norm) * 0.8
    control_signal[side_to_modulate] -= modulation_amount
    return control_signal

In [26]:
x = generate_specific_turning_control(arena, yeast_source, sim, attractive_gain)

In [27]:
x

array([0.99979967, 1.        ])